<a href="https://colab.research.google.com/github/wellecks/llmstep/blob/master/python/colab/llmstep_colab_server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [`llmstep`](https://github.com/wellecks/llmstep) server using Colab notebook
In order to use this notebook, follow these instructions:

0. First, enable GPU by going to `Runtime` -> `Change runtime type` -> `T4 GPU`.

1. Run all the cells in this colab notebook to start your server.

2. In your local environment, set the environment variable `LLMSTEP_HOST` equal to the url printed out in this notebook (for example, https://04fa-34-125-110-83.ngrok.io/)

3. In your local environment, set the environment variable `LLMSTEP_SERVER=COLAB`.

4. Use `llmstep`.


#### VSCode steps (2) and (3)

To set environment variables in VS Code, go to:

- Settings (`Command` + `,` on Mac)
- Extensions -> Lean 4
- Add environment variables to `Server Env`.
- Then restart the Lean Server (`Command` + `t`, then type `> Lean 4: Restart Server`)


Authors: Rahul Saha, Sean Welleck

### Setup `llmstep` code

In [ ]:
!pip install pyngrok
!pip install flask
!pip install transformers
!pip install flask_ngrok

!wget https://github.com/wellecks/llmstep/archive/92990596f91aad7e1323a8985d46a488ce8aef57.zip
!unzip 92990596f91aad7e1323a8985d46a488ce8aef57.zip

import sys
sys.path.append('./llmstep-92990596f91aad7e1323a8985d46a488ce8aef57/python/')

import server

### Run : load the model and start the server

The cell prints out the public URL, for instance: https://04fa-34-125-110-73.ngrok.io

**Add this URL as a `LLMSTEP_HOST` environment variable in your local environment.**

In [ ]:
from pyngrok import ngrok

# Open a tunnel
port = 80
public_url = ngrok.connect(port)

# Setup the config (modify the options here if you want)
config = {
    'LLMSTEP_MODEL': 'wellecks/llmstep-mathlib4-pythia2.8b',
    'LLMSTEP_TEMPERATURES': [0.5],
    'LLMSTEP_NUM_SAMPLES': 5,
    'LLMSTEP_PROMPT': server.llmstep_prompt,
    'LLMSTEP_HOST': '',
    'LLMSTEP_PORT': port
}

# Load the model
print("Loading model...")
model, tokenizer = server.load_hf(config['LLMSTEP_MODEL'])
print("Loading model done.")

# Start the server
httpd = server.LLMStepServer(
    model, tokenizer, server.hf_generate, config
)

print('Your public url is:\n%s\n\nSet LLMSTEP_HOST to this url.' % public_url.public_url)
print('Server started')
httpd.serve_forever()